In [1]:
import numpy as np 
import pandas as pd 

import os

df = pd.read_csv('CarsSet/number_of_samples.csv')
df





,Unnamed: 0,Number of samples for training,Number of samples for testing
0,0,269,57


In [9]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import xml.etree.ElementTree as ET
import keras
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input
from keras.applications.resnet50 import ResNet50


def load_and_preprocess_dataset():
    x_train = []
    y_train = []
    data = os.listdir('CarsSet/DATA/DATA/train')
    for image in data:
        if(image.endswith('.jpg')):
            # Load the image
            img = cv2.imread(os.path.join('CarsSet/DATA/DATA/train', image))
            # Resize the image to 224x224
            img = cv2.resize(img, (224, 224))
            x_train.append(img)
        else:
            # Load the XML annotation file
            tree = ET.parse(os.path.join('CarsSet/DATA/DATA/train', image))
            root = tree.getroot()
            bndbox = root.find('.//bndbox')
            bbox = []
            # Extract the xmin, ymin, xmax, and ymax values from the bndbox element
            bbox.append(int(bndbox.find('xmin').text))
            bbox.append(int(bndbox.find('ymin').text))
            bbox.append(int(bndbox.find('xmax').text))
            bbox.append(int(bndbox.find('ymax').text))
            y_train.append(bbox)

    x_val = []
    y_val = []
    data = os.listdir('CarsSet/DATA/DATA/validate')
    for image in data:
        if(image.endswith('.jpg')):
            # Load the image
            img = cv2.imread(os.path.join('CarsSet/DATA/DATA/validate', image))
            # Resize the image to 224x224
            img = cv2.resize(img, (224, 224))
            x_val.append(img)
        else:
            # Load the XML annotation file
            tree = ET.parse(os.path.join('CarsSet/DATA/DATA/validate', image))
            root = tree.getroot()
            bndbox = root.find('.//bndbox')
            bbox = []
            # Extract the xmin, ymin, xmax, and ymax values from the bndbox element
            bbox.append(int(bndbox.find('xmin').text))
            bbox.append(int(bndbox.find('ymin').text))
            bbox.append(int(bndbox.find('xmax').text))
            bbox.append(int(bndbox.find('ymax').text))
            y_val.append(bbox)

    return np.array(x_train), np.array(y_train), np.array(x_val), np.array(y_val)


# Load and preprocess the dataset
X_train, Y_train, X_val, Y_val = load_and_preprocess_dataset()

# Define the model
input_shape = X_train.shape[1:]
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(4, activation='linear')(x)
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
model.fit(X_train, Y_train, batch_size=32, epochs=20, validation_data=(X_val, Y_val))

# Evaluate the model
score = model.evaluate(X_val, Y_val, verbose=0)
print("Validation loss:", score[0])
print("Validation MSE:", score[1])




c:\Users\joshu\AppData\Local\Programs\Python\Python37\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Train on 269 samples, validate on 28 samples
Epoch 1/20
269/269 [==============================] - 86s 319ms/step - loss: 45546.8794 - mean_squared_error: 45546.8794 - val_loss: 34963.3633 - val_mean_squared_error: 34963.3633
Epoch 2/20
269/269 [==============================] - 78s 289ms/step - loss: 13690.7702 - mean_squared_error: 13690.7702 - val_loss: 25692.2227 - val_mean_squared_error: 25692.2227
Epoch 3/20
269/269 [==============================] - 84s 314ms/step - loss: 6097.8712 - mean_squared_error: 6097.8712 - val_loss: 31900.2676 - val_mean_squared_error: 31900.2676
Epoch 4/20
269/269 [==============================] - 83s 310ms/step - loss: 4914.3812 - mean_squared_error: 4914.3812 - val_loss: 74769.8125 - val_mean_squared_error: 74769.8125
Epoch 5/20
269/269 [==============================] - 79s 295ms/step - loss: 3949.8510 - mean_squared_error: 3949.8510 - val_loss: 20543.1445 - val_mean_squared_error: 20543.1445
Epoch 6/20
269/269 [==============================] - 81

In [32]:
model.save_weights("model_20.tf")

In [40]:
# Load the image you want to draw bounding boxes on

data = os.listdir('CarsSet/DATA/DATA/test')

# Loop through the image filenames and process each image
for filename in data:
    if(filename.endswith('.jpg')):
        # Load the image
        img = cv2.imread(os.path.join('CarsSet/DATA/DATA/test', filename))
        # Preprocess the image (resize and normalize)
        img = cv2.resize(img, (224, 224))
        image = np.array(img)
        image = np.expand_dims(image, axis=0)  # add batch dimension

        # Use the model to predict the bounding box coordinates
        preds = model.predict(image)
        x1, y1, x2, y2 = preds.astype('int')[0]

        # Draw the bounding box on the image
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Display the image with the bounding box
        cv2.imshow('Image', img)
        cv2.waitKey(0)

# Close all windows
cv2.destroyAllWindows()
